<a href="https://colab.research.google.com/github/ashlin07/ieee-ospi-atm-intrustion-detection-system/blob/atm_instrusion_model/ATM_intrustion_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import numpy as np

In [4]:
NUM_CLASSES = 4

In [5]:
dataset = 'coords_new.csv'
X_dataset = np.genfromtxt(dataset, delimiter=',', dtype='float32', usecols=range(1, (33* 4) + 1))

y_dataset = np.genfromtxt(dataset, delimiter=',', dtype='int32',usecols=0)


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=42)

In [8]:
print(y_dataset[2000:2005])

[1 1 1 1 1]


In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

In [10]:
!pip install tensorflow

In [11]:
model = Sequential(
    [
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=100
)


Epoch 1/100
138/138 [==============================] - 1s 1ms/step - loss: 1.0699
Epoch 2/100
138/138 [==============================] - 0s 2ms/step - loss: 0.5190
Epoch 3/100
138/138 [==============================] - 0s 2ms/step - loss: 0.2921
Epoch 4/100
138/138 [==============================] - 0s 2ms/step - loss: 0.1903
Epoch 5/100
138/138 [==============================] - 0s 2ms/step - loss: 0.1323
Epoch 6/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0981
Epoch 7/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0775
Epoch 8/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0640
Epoch 9/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0517
Epoch 10/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0466
Epoch 11/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0431
Epoch 12/100
138/138 [==============================] - 0s 2ms/step - loss: 0.0349
Epoch 13/100


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (32, 25)                  3325      
                                                                 
 dense_1 (Dense)             (32, 15)                  390       
                                                                 
 dense_2 (Dense)             (32, 4)                   64        
                                                                 
Total params: 3,779
Trainable params: 3,779
Non-trainable params: 0
_________________________________________________________________


In [13]:
p_preferred = model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

138/138 [==============================] - 0s 1ms/step
two example output vectors:
 [[  5.528055   -4.138232   -8.215636  -21.801899 ]
 [  3.962799   -7.2146935 -25.490604   -7.73548  ]]
largest value 45.205017 smallest value -42.15126


In [14]:
print(y_train[100])
print( f"{p_preferred[100]}, category: {np.argmax(p_preferred[100])}")

2
[-9.7928295 -4.4929233  9.011928  -4.654128 ], category: 2


In [15]:
count=0
for i in range(3000):
    m, n = X_dataset.shape
    random_index = np.random.randint(m)
    prediction = model.predict(X_dataset[random_index].reshape(1,33*(4)),verbose=None)
    prediction_p = tf.nn.softmax(prediction)
    yhat = np.argmax(prediction_p)
    if(yhat!=y_dataset[random_index]):
        count+=1

print(count)

0


In [16]:
m, n = X_dataset.shape
random_index = np.random.randint(m)
prediction = model.predict(X_dataset[random_index].reshape(1,33*(4)))
prediction_p = tf.nn.softmax(prediction)
yhat = np.argmax(prediction_p)

1/1 [==============================] - 0s 24ms/step


In [17]:
print( f"{display_errors(model,X,y)} errors out of {len(X)} ")

NameError: name 'display_errors' is not defined

In [ ]:
model.save('model_intrusion.h5')
